In [7]:
#Imported Libraries
import numpy as np # Used for generating the Dataset Arrays
import os # Used for generating file names and dictionarys. 
from os.path import isfile, join # Used for generating file names and dictionarys. 
import math # Used for min and max Y axis values. 
import h5py # Used for saving the output HDF5 File. 
from datetime import date # Date used in output file additional information. 
from scipy import signal as signal # Used for Filter. 


Version = 2         # version number of the script


#Setting base directory for Python Script

#os.chdir(os.path.dirname(os.path.abspath(__file__))) # UN-HASH WHEN CONVERTED TO .PY SCRIPT. Used to set base directory in python terminal to enable relative input and output folder names to be used. 


## FUNCTIONS

def InputVariableNumberChecker(Value,Defaultvalue): # Primarily used with a filter window parameter with a default arugment. Checks that the value inputted is a integer. If not the default argument is used. 
    try: # try to see if the value can be converted to a integer. If it can the function just returns the input value
        int(Value) 
        return Value    
    except ValueError: # if the above returns an error it outputs the default argument value as the function output. 
        print(f"Suitable value not inputted. Default of {Defaultvalue} is used instead")
        return Defaultvalue

def NumberCheck(Input): # Checks that the value inputted it actually an integer. If not continues to ask until a Integer is inputted. 
    
    for ListItem in Input: # cycles through all the seperate elements in the list argument
        try:
            int(ListItem) # try to see if the value can be converted to a integer. If it can the function cycles to the next item in the list. 
            
        except ValueError: # if the above returns an error it asks again for the user to input a correct number. It then asks for a whole new list and re-initilizes the checking function. It then outputs the updated/corrected input list. 
            ColumnInputString = input("A non-integer or space was inputted. Numbers seperated by spaces only! Try again ->   ")
            ColumnsToProcess = ColumnInputString.split(" ")
            Input = NumberCheck(ColumnsToProcess) # performs the NumberCheck function again to confirm that a number was inputted the second time. 
    
# I wonder if this really works if taken to the worst case. First item fails. It asks for an updated list. It then completes with a new list. Does it then cycle to the oringial second value in the first un-updated list? 

    return Input

def SamplePeriodFunction(File): #returns the sample time by reading the CSV time stamps and calculating the difference. scaled to seconds. Float value. 
    x = open("./%s"%(File), "r") # opens the csv file for reading
    SP = [] # defining sampledata as a list for the initial extraction of the data
    count = 0 # Defining the count for the while loop
    while count<=2: # while the count is below 2 do the following. This function results in SP containing two time entries within a list. 
        if count==0: # if the count is still 0 - ie the first line 
            x.readline() # read the line (so that python moves onto the next line on the next loop)
            count+=1 # add 1 to the count so that the else function is used next time
        else:
            count+=1 # add to the count
            A = x.readline() # read the next line and assign it to the string A
            n = 0 # another counter for the FOR loop - used for indexing purposes in the .append line below. 
            for a in A: # for every character in the string A do the following
                if a==";": # if the character is the delimiter use the n counter to scrape the previous time data away
                    SP.append(int(A[n-9:n-3])) # extracts the time data away, note the indexing may need changing later on. Int() changes the string to an int. adds the value to the ST list defined above.
                    break # stop the loop from happening again - function errors if run again. 
                else:
                    n+=1 # add one to the counter for the next character to be accessed
            
    x.close() # close the file. important. 
    SP = SP[1] - SP[0] #calculates the difference. Converts list to int
    SP = float(SP / 1000000) #scaled to seconds. converts int to float value.  
    return SP # return the float ST as the output of the function. 

def TimeArray(): # Generating an time array for the first column in the DataFrame. Finds the Sample Period, Length of the data array (in terms of cells) and then generates a Time Array based of those values. 
    global TargetFile
    global FileList
    global SamplePeriod
    global CsvDelim
    global TotalTimeLength

    
    SamplePeriod = SamplePeriodFunction(FileList[TargetFile]) # Finds the Sample Period using the SamplePeriodFunction
    SampleLength = np.loadtxt("./%s"%(FileList[TargetFile]), delimiter=CsvDelim, usecols=1, skiprows=1) # load the data from the csv
    A = SampleLength.shape # measure the shape to understand the number of y values
    TotalTimeLength = round((A[0]*SamplePeriod),4) # turning the number of Y values into a time
    #The following line generates the array between 0 and the time calculated above at an increment of the calculated sample period. 
    TimeArrayValues = np.arange(0,TotalTimeLength,SamplePeriod).round(decimals = 4) # Note the use of Round. This rounds the numbers generated for the time to a maximum of 4 decimal places. Stops float errors. 
    

    return TimeArrayValues

def Filtering(Data): # Applies a Savitzky - Golay filter to the waveform. Primarily to remove zero order hold sampling artifacts. 
    
    global Time
    global FilterTriggerWindowWidth
    global MovingAverageWindowSize
    
    Data = signal.savgol_filter(Data, MovingAverageWindowSize, 1, mode="mirror", deriv=0) # no phase shift filter to smooth out quantisation/sampling hold steps. 
    
    return Data

def BlankColumnProcessor(): # If the column to be processed is deemed blank a array of NaN values is generated. In reality this is unlikely to be used due to a length checker in the columnstoprocess variable. 
    
    
    global OutputDataFrame
    global DataArray
    global ChannelDescriptions
    global YAxisScalingValuesExport

    ProcessLabel = "NO DATA"
     
    Length = int(DataArray.shape[0]) # find length of currently generated DataArray to find the number of values required. 
        
    ColumnData = np.empty(Length) # generate the Array.  
    ColumnData[:] = np.nan # Populate with NaN values

    DataArray = np.column_stack((DataArray,ColumnData)) # Add the existing DataArray

    YAxisScalingValuesExport.append(0) # add values to the Y scaling list that is exported with the HDF5 File
    YAxisScalingValuesExport.append(0)
    ChannelDescriptions.append(ProcessLabel) # Add the string "No Data" to the label llist that is exported with the HDF5 file

def ColumnProcessor(Column): # Processing each column, in the order identifed in the input and adding it to the DataFrame, Y value axis values and Labels as Column Headers generated and added to the respective list to be exported with the HDF5 File. 
    
    global TargetFile
    global FileList
    global ScalingDictionary
    global DataArray
    global FilteringEnabled
    global CsvDelim
    global YAxisScalingValuesExport
    global ChannelDescriptions
    global ChannelUnits
    global OriginalHeader
   
    # User Inputs
    ProcessLabel = input(f"\nInput a label name/description for the data in Column {Column}, -  {OriginalHeader[Column-1][1:-1]}   ->   ")
    ProcessUnit = input(f"Define the unit of measurement for {ProcessLabel} -> ")

    # Loading the relavent channal dataframe. 
    Column = int(Column)
    ColumnData = np.loadtxt("./%s"%(FileList[TargetFile]), delimiter=CsvDelim, usecols=Column, skiprows=1) # load the data from the csv

    # Apply the Savitzky - Golay filter function if required.   
    if FilteringEnabled == "YES" or FilteringEnabled == "Yes" or FilteringEnabled == "yes":
        ColumnData = Filtering(ColumnData)  

    # Round the floating value numbers to 4dp.     
    ColumnData = ColumnData.round(decimals = 4)
    
    # Axis Scaling Values - Automatically estimate and then asks the user to agree or update. 
    
    YminValue = round(float(np.nanmin(ColumnData[ColumnData != -np.inf])),4)  # finds the min value in the channel dataset but only looks at values that are not infinate. 
    YmaxValue = round(float(np.nanmax(ColumnData[ColumnData != np.inf])),4)     # Finds the Max Value
        
    YSpan = round((YmaxValue - YminValue),4) # finds the span of Y values (not including inf) from the min and max values
    YAxisSpacingPercentEitherSide = 10 # variable to define the buffer either side ## MIGHT WANT TO ADJUST THIS
    ValueBuffer = round(YSpan * (YAxisSpacingPercentEitherSide/100), 4) # calcultes this percentage as an actual value relative to the span. 
    Ymin = math.floor(YminValue - ValueBuffer) # finds the new min value, rounded down to the nearest number. 
    Ymax = math.ceil(YmaxValue + ValueBuffer) # finds the nex max value, rounded up to the nearest number. 

    
    AxisChecker = input(f"For Channel {ProcessLabel} the calculated Y axis min/max values are {Ymin} and {Ymax}, if you want to change these return any value other than a return ->   ")
    if len(AxisChecker) != 0:
        Ymin = round(float(input(f"The automatic MIN Y axis value for data in Channel {ProcessLabel} is {Ymin}. Type in new value->   ")),2)
        Ymax = round(float(input(f"The automatic MAX Y axis value for data in Channel {ProcessLabel} is {Ymax}. Type in new value->   ")),2)

    # Adds the additional information collected to the relavent lists. The idea is the index position for a given channel data is the same between all lists and the data array itself. 

    YAxisScalingValuesExport.append(Ymin)
    YAxisScalingValuesExport.append(Ymax)
    ChannelDescriptions.append(ProcessLabel)
    ChannelUnits.append(ProcessUnit)

    # Adds the Array to the existing DataArray as an additional column. 

    DataArray = np.column_stack((DataArray,ColumnData))
    
    



In [8]:
# FILE SELECTION - CSV Data Location / List. Generates list of files within the defined folder. 

# Constants

CsvDelim = ";" # delimter within cvs
InputFolder = "./Input_CSV_Files" # input file path
HDF5OutputFilePath = "Processed_HDF5_Files" # Output Folder

# Code

print(f"Files located within {InputFolder}") # Header
FileList = [f for f in os.listdir(InputFolder) if isfile(join(InputFolder, f))] # look through the folder specified in the variable above and generate a list of all the files within that folder. 
InputFolder = InputFolder[2:] + "/" # conerting the file path input constant to work correctly with susequent open/import requests
FileList = [InputFolder + s for s in FileList] # Adds the filepath to the file name so it can be easily used by following import/open functions to all the values in the list. (for each list element in FileList (S), add the element to the end of the ImportFolder string. Return as a list element)
FileListCounter = 1 # Counter for print loop below
for I in FileList: # printing the csv datasets within the folder - makes it easier to select the desired file
    print(f"File {FileListCounter} = {I[16:]}")
    FileListCounter +=1

Files located within ./Input_CSV_Files
File 1 = TestFile1.csv
File 2 = TestFile2.csv
File 3 = TestFile3.csv


In [9]:
TargetFile = input("\nWhich file to process? Input a integer value between 1 & %s  ->   "%(len(FileList)))  # asks the question about which file to process. Wants an input. 

#TargetFile = "9" #Default for test purposes

TargetFile = NumberCheck(TargetFile) # Checking the value inputted is a number. 
TargetFile = int(TargetFile)-1 # the minus one is to convert the inputted value (converted into an int), 1, 2 etc into a index position for the list. 
if TargetFile > (len(FileList)-1) or TargetFile == -1: # Checking that the value is actually a legit value, ie that there is a list index position for the number inputted or that 0 hasnt been inputted
    print("The number entered is greater than the number of dataset files available")
    input("Press any key to exit  ") # A way to pause the program and ensure the user understands why the program is about to stop. 
    quit() # Stops the program. 


OriginalFile = open("./%s"%(FileList[TargetFile]), "r") # opens the target file
OriginalHeader = OriginalFile.readline() # read a line from the document. Python behind the scenes knows the read the next line when its next run
OriginalHeader = OriginalHeader.strip() # the strip removes the return for new line (/n) from the string.
OriginalFile.close() # close the file. must do. 
OriginalHeader = OriginalHeader.split(";") # split the string into a list using the sepeartor ;
OriginalHeader.pop(0) # deletes the first element in the Original Header list - ie the TIME column. 
OriginalHeaderCounter = 1 # counter for printing loop below. 
print("") # Generates a space in the terminal - used to help userability. 
for a in OriginalHeader:
    print(f"Channel {OriginalHeaderCounter} = {a[1:-1]}") # f function to print two variables inline
    OriginalHeaderCounter +=1 # counter for channel number in print function. 
#print("") # Ganerates a space in the terminal. Unsure if required. 


Channel 1 = 1__1_2500_044_Engine_Speed__ECU_
Channel 2 = 1__1_0181_001_P_Charge_Mix_A
Channel 3 = 1__1_4540_007_Fuel_Flow


In [10]:
# Defining the columns to process. Checks that the inputted values are within the number of potential selectable columns. 

ColumnInputString = input("\nWhich channels to process - %s Channels to select from - State channel number in the desired order. \nChannel 0 results in a channel array of NaN's (Use if your missing a channel in the oringial file). Input integers and spaces only - eg. 1 21 3..... for exmaple ->   "%(len(OriginalHeader))) # input for the desired columns to import in the order desired. 
#ColumnInputString = "5" # Default for testing purposes
ColumnsToProcess = ColumnInputString.split(" ") # splitting string full of numbers (hopfully) into a list. 
ColumnsToProcess = NumberCheck(ColumnsToProcess) # checking that all the values in the list are actually numbers

for CP in ColumnsToProcess: # checking that the numbers within that list are actual channels
    if int(CP) > len(OriginalHeader): #or int(CP) == 0: # if the value in the list is greater than the length of potential columns program stops in the same way as above.
        print("The number entered is greater than the number of datasets available")
        input("Press any key to exit  ")
        quit()

# Is filtering required. If so define the filter window. 

FilteringEnabled = input("\nDo you want to apply Savitzky Golay Filtering to the sample if Zero-Order sampling values are detected? Yes or No ->   ")

if FilteringEnabled == "YES" or FilteringEnabled == "Yes" or FilteringEnabled == "yes":
    #FilterTriggerWindowWidth = input("Define the window width you want to try identify Zero-Order sampling across. Default is 20. ->   ")    # This was previously used to try and identify Zero Order Sampling artifacts. Not used. User decides if to filter or not. 
    #FilterTriggerWindowWidth = InputVariableNumberChecker(FilterTriggerWindowWidth, 20)
    MovingAverageWindowSize = input("Define the window size for the Savitzky Golay Filter. Default is 60. ->   ")
    MovingAverageWindowSize = InputVariableNumberChecker(MovingAverageWindowSize, 60)


  

In [11]:
DataArray = TimeArray() # Generates the Time array using the function defined above

In [12]:
# Setting up additional information lists

YAxisScalingValuesExport = []
ChannelDescriptions = []
ChannelUnits = []

# Main processing loop. For each column defined by the user to be processed (in the order inputted) scrap the data from the input csv and add it to the DataFrame. 

for ColumnItem in ColumnsToProcess: 
    ColumnItem = int(ColumnItem)
    if ColumnItem == 0:
        BlankColumnProcessor()
    else:
        ColumnProcessor(ColumnItem) # The function is defines above. 

#Delete the time column from the data array. - Saves space. Not needed considering the sample time is sent wtih the HDF5 file. 
DataArray = np.delete(DataArray,0,1) 


# Generating / Collecting the desired file output name. Either takes a user inputted string or the original filename. 

DefaultOutputFileName = FileList[TargetFile]
DefaultOutputFileName = DefaultOutputFileName[16:-4]

OutputFileName = input("Type in the desired output File name. %s.hdf5 is the default filename if no string is inputted ->   "%(DefaultOutputFileName))
if len(OutputFileName) == 0: # Used to detect if nothing was inputted, ie to use the default value. 
    OutputFileName = DefaultOutputFileName
    


In [13]:

#Additional Info to save with the file as aditional information. 

ScriptVersionInfo = "This file was generated with Version %s of the DiaSys CSV HDF5 Processor Script on %s."% (Version, date.today().strftime("%d %B %Y"))
OriginalImportFile = "This dataset was processed from the oringial file - %s"%(FileList[TargetFile][16:])
if FilteringEnabled == "YES" or FilteringEnabled == "Yes" or FilteringEnabled == "yes":
    FilteringInfo = "A Savitzky Golay Filter was applied to this dataset using a window size of %s."%(MovingAverageWindowSize)
else:
    FilteringInfo = "No Filter was applied to this dataset"
TimeInfo = "This recording is %s seconds long with a sample period of %s seconds."%(TotalTimeLength,SamplePeriod)

FileDescription = [TimeInfo,FilteringInfo,OriginalImportFile,ScriptVersionInfo]

print ("\n---- Additional Information saved within the HDF5 File ----")
for Instance in ChannelDescriptions:
    print(Instance)
for Instance in FileDescription:
    print(Instance)

# Saving the HDF5 File. 

SaveFile = h5py.File("%s/%s.hdf5"% (HDF5OutputFilePath, OutputFileName), "w") # Generating the hdf5 file
SaveFile["Data"] = DataArray #saving the Dataset variable to a part of the file called Data 
SaveFile["Data"].attrs["Info"] = FileDescription # defining an attribute of that saved file with the key Info - File information. 
SaveFile["Data"].attrs["ChannelDescriptions"] = ChannelDescriptions # exported in sequence
SaveFile["Data"].attrs["ChannelUnits"] = ChannelUnits # exported in sequence
SaveFile["Data"].attrs["YScales"] = YAxisScalingValuesExport # exported in sequence (A Ymin, A Ymax, B Ymin, B Ymax.....)
SaveFile["Data"].attrs["SamplePeriod"] = SamplePeriod # Exported as int value. 
SaveFile["Data"].attrs["SampleLength"] = TotalTimeLength # Exporting an float of the length of the recording (in seconds)
SaveFile.close() # closing the file. 


---- Additional Information saved within the HDF5 File ----
Engine Speed
A Bank Inlet Manifold
Fuel Flow
This recording is 62.9 seconds long with a sample period of 0.005 seconds.
No Filter was applied to this dataset
This dataset was processed from the oringial file - TestFile1.csv
This file was generated with Version 2 of the DiaSys CSV HDF5 Processor Script on 13 August 2024.
